<a href="https://colab.research.google.com/github/karagol-taner/ollama-web-interfaces-with-colab-main/blob/main/LLM_model_with_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# Kill any existing Ollama processes
!pkill -f ollama

# Start Ollama in the background
!nohup ollama serve > /dev/null 2>&1 &

!ollama pull model-name

pulling manifest ⠋ verifying sha256 digest 
writing manifest 
success 


In [ ]:
# Kill any existing Ollama processes
!pkill -f ollama

# Start Ollama in the background
!nohup ollama serve > /dev/null 2>&1 &

In [ ]:
!ollama run model-name # you can use this on terminal of colab too

**For gradio live chatbox:**

In [ ]:
# Kill any existing Ollama processes
!pkill -f ollama

# Start Ollama in the background
!nohup ollama serve > /dev/null 2>&1 &

In [ ]:
!pip install gradio

In [ ]:
import requests, json

import gradio as gr


model = 'model-name' #You can replace the model name if needed
context = []



import gradio as gr

#Call Ollama API
def generate(prompt, context, top_k, top_p, temp):
    r = requests.post('http://localhost:11434/api/generate',
                     json={
                         'model': model,
                         'prompt': prompt,
                         'context': context,
                         'options':{
                             'top_k': top_k,
                             'temperature':top_p,
                             'top_p': temp
                         }
                     },
                     stream=False)
    r.raise_for_status()


    response = ""

    for line in r.iter_lines():
        body = json.loads(line)
        response_part = body.get('response', '')
        print(response_part)
        if 'error' in body:
            raise Exception(body['error'])

        response += response_part

        if body.get('done', False):
            context = body.get('context', [])
            return response, context



def chat(input, chat_history, top_k, top_p, temp):

    chat_history = chat_history or []

    global context
    output, context = generate(input, context, top_k, top_p, temp)

    chat_history.append((input, output))

    return chat_history, chat_history
  #the first history in return history, history is meant to update the
  #chatbot widget, and the second history is meant to update the state
  #(which is used to maintain conversation history across interactions)

#########################Gradio Code##########################
block = gr.Blocks()


with block:

    gr.Markdown("""<h1><center> model-name </center></h1>
    """)

    chatbot = gr.Chatbot()
    message = gr.Textbox(placeholder="Type here")

    state = gr.State()
    with gr.Row():
        top_k = gr.Slider(0.0,100.0, label="top_k", value=40, info="Reduces the probability of generating nonsense. A higher value (e.g. 100) will give more diverse answers, while a lower value (e.g. 10) will be more conservative. (Default: 40)")
        top_p = gr.Slider(0.0,1.0, label="top_p", value=0.9, info=" Works together with top-k. A higher value (e.g., 0.95) will lead to more diverse text, while a lower value (e.g., 0.5) will generate more focused and conservative text. (Default: 0.9)")
        temp = gr.Slider(0.0,2.0, label="temperature", value=0.8, info="The temperature of the model. Increasing the temperature will make the model answer more creatively. (Default: 0.8)")


    submit = gr.Button("SEND")

    submit.click(chat, inputs=[message, state, top_k, top_p, temp], outputs=[chatbot, state])


block.launch(debug=True)
